# Introduction
From previous 2D simulations of different bootstrap methods to construct SCB, we saw that the nonparametric bootstrap with Radmacher multipliers and t-standardization is the best. This is true in Gaussian noise setting and the t3 noise setting. However, the fMRI data may not follow Gaussian or t3. 

Here, we perfrom resting-state validation to the bootstrap method and the inverse set method. 



**Data**

We used resting-state data from 1,000 Functional Connectomes Project. 

**First-level Analysis**

In FSL, a first-level analysis has been conducted independently for each subject. 




In [ ]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt 
import SimuInf
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize
import random
import sys
import os
import math
import scipy.stats
from tabulate import tabulate
import time
import joblib

#import sanssouci as ss

import pyperm as pr
from nilearn.plotting import plot_img, plot_stat_map, view_img
from nilearn.image import get_data, load_img, new_img_like

# Data Import 

Now we read the data, which were previously downloaded into the folder from FSL folder in http://tinyurl.com/clusterfailure

We use the Beiging.zip dataset since it has the most particiants, which will reduce dependence between resampled sub-samples. The folder has contrast maps from 198 subjects under various task paradim and smoothing parameters. Here, we use boxcar10_smoothing_4mm, where boxcar10 means the block activity with 10-s on-off. 


In [ ]:
# Real data directory
data_dir = os.path.join(os.getcwd(),'data')
beijing_data_dir = os.path.join(os.getcwd(),'data','Beijing')
bold_files_total =  os.listdir(beijing_data_dir)
print('# of total files in the folder (various settings):', len(bold_files_total))
bold_files = [os.path.join(beijing_data_dir, file) for file in bold_files_total if 'boxcar10_smoothing_4mm' in file]
print('# of subjects in Beijing dataset(one setting):', len(bold_files))
print(bold_files[:3])

In [ ]:

bold_files1 = [os.path.join(beijing_data_dir, file) for file in bold_files_total if 'Event3_smoothing_4mm' in file]
print('# of subjects in Beijing dataset(one setting):', len(bold_files1))
print(bold_files1[:3])

In [ ]:
## raw constrat image from the first subject 
plot_img(bold_files[0], colorbar = True)
plot_stat_map(bold_files[0], colorbar=True)
print("the shape of the first bold file:", get_data(bold_files[0]).shape)

In [ ]:
## raw constrat image from the first subject 
plot_img(bold_files1[0], colorbar = True)
#plot_stat_map(bold_files1[0], colorbar=True)
print("the shape of the first bold file:", get_data(bold_files1[0]).shape)

In [ ]:
plt.hist(get_data(bold_files[0]).flatten(), bins=200, density = False)
plt.show()
np.sum(get_data(bold_files[0])==0)

First we apply the mask, which will reduce the image size and focus only on areas in the brain and apply some smoothing.

Here we use the MNI mask, which is of shape (91, 109, 91)

In [ ]:
mask = data_dir + '\\MNImask.nii' #MNI mask 
plot_img(mask)
print("the shape of the MNI mask:", get_data(mask).shape)
print("number of voxels inside the MNI mask", np.sum(get_data(mask)))

In [ ]:
bold_data = get_data(bold_files)

In a contrast map, if the voxel is exactly 0, then that is outside of the mask. Each subject's brain is different so the mask should be different. Here, we apply a stricter mask, which is the intersection of each individual's mask in our sample. That is, any voxel with at least one zero will be excluded. If the mask is too big, then for some voxels, the majority of the voxels are 0 and the sd will be 0.

In [ ]:
bold_data1 = get_data(bold_files1)

In [ ]:
# create a mask, which is the intersection of the voxels inside brain among all people
# this mask is smaller than the MNI mask
print(bold_data.shape)
mask1 = np.all(bold_data, -1)*1
print(mask1.shape)
print('number of voxels in the self-created mask:', np.sum(mask1))

# covert the np array into Nifti1Image
mask1_img = new_img_like(mask, mask1)
print(type(mask1_img))
plot_img(mask1_img, colorbar=True)

In [ ]:
# intersection with the MNI mask: self-created one does not have those holes because the in first-level analysis, the mask they used does not have holes
# there will be always signal and those 0s are manually put based on a mask
mask_final = np.all(np.array([get_data(mask), mask1]), 0)
print(mask_final.shape)
print('number of voxels in the final mask:', np.sum(mask_final))

# covert the np array into Nifti1Image
mask_final_img = new_img_like(mask, mask_final)
plot_img(mask_final_img, colorbar=True)


In [ ]:
mask_final1  = np.all(np.array([get_data(mask), np.all(bold_data1, -1)*1]), 0)
print(mask_final1.shape)
print('number of voxels in the final mask:', np.sum(mask_final1))
mask_final_img1 = new_img_like(mask, mask_final1)

In [ ]:
from nilearn.input_data import NiftiMasker
# no additional smotthing since contrast maps were already smoothed
fwhm = 0 # Set the smoothness parameter (in mm)
# this creates a NiftiMasker class, if mask_img is not prvided, will compute the mask in the fit step
# mask_img needs to be Niimg-like object so it cannot be the array.# rerun!
masker = NiftiMasker(smoothing_fwhm=fwhm, mask_img=mask_final_img).fit()
data = masker.transform(bold_files).transpose()
print(data.shape)

In [ ]:
from nilearn.input_data import NiftiMasker
# no additional smotthing since contrast maps were already smoothed
fwhm = 0 # Set the smoothness parameter (in mm)
# this creates a NiftiMasker class, if mask_img is not prvided, will compute the mask in the fit step
# mask_img needs to be Niimg-like object so it cannot be the array.# rerun!
masker1 = NiftiMasker(smoothing_fwhm=fwhm, mask_img=mask_final_img1).fit()
data1 = masker1.transform(bold_files1).transpose()
print(data1.shape)

In [ ]:
data_mean = np.mean(data, 1)
print('mean of data_mean:', np.mean(data_mean), 'max of data_mean:', np.max(data_mean), 
      'min of data_mean:', np.min(data_mean))
# masker.inverse_transform: Transform the 2D data matrix back to an image in brain space.
## This step only performs spatial unmasking, without inverting any additional processing performed by transform, such as temporal filtering or smoothing.
## using this unmasked version only for plotting. the analysis was done on maksed data. 
## get_data: Get the image data as a numpy.ndarray.
data_mean_3d = get_data(masker.inverse_transform(data_mean))


print("number of voxels in the mask:", np.prod(get_data(mask_final_img).shape), 'from shape:', get_data(mask_final_img).shape)
print("number of voxels being 1 in the mask:", np.sum(get_data(mask_final_img)))

# sanity check 
# raw 3D image has the largest shape
print("number of voxels in raw contrast image:", np.prod(get_data(bold_files[0]).shape), 'from shape:', get_data(bold_files[0]).shape)
# this is 3d unmasked(with inverse transform), so it has voxels outside the mask and has the same shape as the mask
print("number of voxels in unmasked data_mean_3d:", np.prod(data_mean_3d.shape), 'from shape:', data_mean_3d.shape)
# this is 1d masked data mean, so it only has voxels inside the mask
print("number of voxels in masked data_mean:", np.prod(data_mean.shape))
plt.imshow(data_mean_3d[:,:,23])
plt.colorbar()
plt.show()

# SCB construction 

Now construct SCB with t-bootstrap with rademacher multilpliers. This is done on the masked data of shape (228433*77). 

From the  3D contrast maps -> sample without replacement to get 20 subjects(this is a subsample)  -> construct SCB 
repeat above 1000 times to calculate the simultaneous covera ratege (the truth is all 0).

In [ ]:
# update
def get_subsample(data, subsample_size=20):
    dim = data.shape
    n_subj = dim[-1]
    data_out = data[..., np.random.choice(n_subj, subsample_size, replace=False)]
    return data_out

In [ ]:
from itertools import product
# with a large subsample size, there is more dependence 
setting_df_resting = pd.DataFrame({'subsample_size': [10,20,30,40,50]})
print(setting_df_resting)
method_df_resting =  pd.DataFrame(product(['res'], ['multiplier'], ['t'], ['r']), columns=['boot_data_type', 'boot_type', 'standardize', 'multiplier'])
print(method_df_resting)

In [ ]:
from SimuInf.simulation import scb_cover_rate_multiple
scb_cover_rate_multiple(setting_df_resting, method_df_resting,
                              data_in = data, 
                      m_sim=10, alpha=0.05,
                      m_boots=10)

In [ ]:
resting_result = scb_cover_rate_multiple(setting_df_resting, method_df_resting,
                              data_in = data, 
                      m_sim=1000, alpha=0.05,
                      m_boots=1000)
#joblib.dump(resting_result, 'results/resting_result_boxcar10_smoothing_4mm')

In [ ]:
resting_result

# Confidence set validation

Now get the simultaneous coverage rate for different levels of c. Based on the theory, inverting the SCB gives valid inference for all c, here, with a finite number of c, the simultaneous coverage rate should be larger. 
We use the signal from 
https://github.com/sjdavenport/StatBrainz/tree/main/BrainImages/Volume/fullmean.nii

In [ ]:
data_sim_mu_path = os.path.join(os.getcwd(),'data','fullmean.nii')
data_sim_mu_raw = get_data(data_sim_mu_path)

In [ ]:
print(data_sim_mu_raw.shape)
# there are nan values 
print(np.max(data_sim_mu_raw), np.min(data_sim_mu_raw))
print(np.sum(np.isnan(data_sim_mu_raw)))

In [ ]:
# make the background black
plot_img(data_sim_mu_path, colorbar=True)

In [ ]:
# transform(imgs, counfounds=None, sample_mask=None): apply mask, spatial and temporal preprocessing. 
# Note, transfrom() also changes the size of the image to match the size of the mask, which is smaller
## bold files here already are temporally processed 
data_sim_mu = masker.transform(data_sim_mu_path).transpose()
print(data_sim_mu.shape)

# create new data: resting state + mean 
data_sim = data + data_sim_mu
print(data_sim.shape)

In [ ]:
# transform(imgs, counfounds=None, sample_mask=None): apply mask, spatial and temporal preprocessing. 
# Note, transfrom() also changes the size of the image to match the size of the mask, which is smaller
## bold files here already are temporally processed 
data_sim_mu1 = masker1.transform(data_sim_mu_path).transpose()
print(data_sim_mu1.shape)

# create new data: resting state + mean 
data_sim1 = data1 + data_sim_mu1
print(data_sim1.shape)

In [ ]:
from SimuInf.scb import confband
from SimuInf.confset import confset
from SimuInf.plotting import confset_plot, ls_plot
from SimuInf.simulation import scb_cover_rate_multiple

In [ ]:
# construct confidence sets based on a list of thresholds
# should not round the level c to avoid repeats 
thresholds_ls = [np.linspace(-20, 20, num=n_threshold) for n_threshold in [5,10,50,100,1000]]
print(thresholds_ls[3])

In [ ]:
test = scb_cover_rate_multiple(setting_df_resting.iloc[:1], method_df_resting,
                              data_in = data_sim1, 
                              mu = data_sim_mu1.reshape(-1),
                      m_sim=10, alpha=0.05,
                      m_boots=10, thresholds_ls = [np.linspace(-20, 20, num=n_threshold) for n_threshold in [2000]])
print(test)

In [ ]:
scb_cover_rate_multiple(setting_df_resting.iloc[:1], method_df_resting,
                              data_in = data_sim, 
                              mu = data_sim_mu.reshape(-1),
                      m_sim=10, alpha=0.05,
                      m_boots=10, thresholds_ls = [np.linspace(-20, 20, num=n_threshold) for n_threshold in [2000]])

In [ ]:
confset_result = scb_cover_rate_multiple(setting_df_resting.iloc[1:], method_df_resting,
                              data_in = data_sim, 
                              mu = data_sim_mu.reshape(-1),
                      m_sim=1000, alpha=0.05,
                      m_boots=1000, thresholds_ls = thresholds_ls)
joblib.dump(confset_result, 'results/scb_confset_result2to5')

In [ ]:
confset_result = scb_cover_rate_multiple(setting_df_resting, method_df_resting,
                              data_in = data_sim1, 
                              mu = data_sim_mu1.reshape(-1),
                      m_sim=1000, alpha=0.05,
                      m_boots=1000, thresholds_ls = thresholds_ls)
joblib.dump(confset_result, 'results/scb_confset_result_E3_1to5')

In [ ]:
confset1 = joblib.load('results/scb_confset_result1')
confset2to5 = joblib.load('results/scb_confset_result2to5')
# need to add a column of design E3 and Block
confset_all = pd.concat([confset1.assign(data='block'), confset2to5.assign(data='block'), confset_result.assign(data='event')], ignore_index=True)
print(confset_all)
confset_all.to_excel("confset_all.xlsx")  